In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
import numpy as np

def conv(z, h, omega_lambda, omega_matter):

    H0=100*h
#conversion Mpc --> km
    conv1=1./(1e6*206265*1.496e8)
#conversion de s --> Gyr
    conv2=3600*24*365.25*1e9
    H0=H0*conv1*conv2

    c1=np.sqrt(omega_lambda*(1.+z)**(-3.))+np.sqrt((omega_lambda*(1.+z)**(-3.))+omega_matter)
    ln=np.log(c1/np.sqrt(omega_matter))
    c2=2./(3.*np.sqrt(omega_lambda))
        
    time = (1./H0)*c2*ln

    return time


In [4]:
#Graficamos las gas face-on con el sphviewer
import matplotlib.pyplot as plt
import h5py
import rotation as rot
import sphviewer as sph


h=0.732
a0=1
omega_lambda=0.716
omega_matter=0.1277/(h**2.)
vector = ('00','01','03','09','13','14','18','19','20','21')

snapshot=range(497,496,-1)
for isnap in snapshot:
    snap = h5py.File('/home/ornela/python/ornella/outputs_1/snap_'+str('%03d'%isnap)+'.h5py', 'r')

    a = snap['subhalo_000/Time'].value
    z = (a0/a)-1
    t = conv(z, h, omega_lambda, omega_matter)
    
    print isnap
    for i in range(9,10):
        pstr = snap['subhalo_0'+ str('%s'%vector[i]) +'/Str/Coordinates'].value
        cm   = snap['subhalo_0'+ str('%s'%vector[i]) +'/Center'].value
        vel  = snap['subhalo_0'+ str('%s'%vector[i]) +'/Str/Velocities'].value
        masa = snap['subhalo_0'+ str('%s'%vector[i]) +'/Str/Masses'].value
        r200 = snap['subhalo_0'+ str('%s' %vector[i])+ '/R200'].value
        
        pgas = snap['subhalo_0'+ str('%s'%vector[i]) +'/Gas/Coordinates'].value
        masa_gas = snap['subhalo_0'+ str('%s'%vector[i]) +'/Gas/Masses'].value
        
#------paso las coordenadas respecto al centro de la galaxia------------------
        xstr = (pstr[:,0]-cm[0])/h
        ystr = (pstr[:,1]-cm[1])/h
        zstr = (pstr[:,2]-cm[2])/h
        r = np.sqrt(xstr**2+ystr**2+zstr**2)
       
        xgas = (pgas[:,0]-cm[0])/h
        ygas = (pgas[:,1]-cm[1])/h
        zgas = (pgas[:,2]-cm[2])/h
        r_gas = np.sqrt(xstr**2+ystr**2+zstr**2)
        
#-----------------------masas------------------------------
        masa = masa/h
        masa_gas = masa_gas/h
        
        rgal=0.15*r200
        
        limit = np.where(r<rgal)
        r_sort = np.sort(r[limit])
        r_indice = np.argsort(r[limit])
                
        Mc = np.cumsum((masa[limit])[r_indice])
        Mtot = Mc[-1]
               
        razon = Mc/Mtot
        noventa, = np.where(razon < 0.9)
        cincuenta, = np.where(razon < 0.5)
        
        r90 = r_sort[noventa]
        r50 = r_sort[cincuenta]
        
#esto sirve si quiero hacer una submuestra de mi conjunto de datos
        veloc,=np.where(r<r50[-1])
#escribo las componentes de la velocidad del centro de masa
        vxcm = sum(masa[veloc]*vel[veloc,0])/sum(masa[veloc])
        vycm = sum(masa[veloc]*vel[veloc,1])/sum(masa[veloc])
        vzcm = sum(masa[veloc]*vel[veloc,2])/sum(masa[veloc])
#saco las velocidades de las estrellas respecto del centro de masa de la galaxia
        vx = vel[:,0] - vxcm
        vy = vel[:,1] - vycm
        vz = vel[:,2] - vzcm
        e1x,e2x,e3x,e1y,e2y,e3y,e1z,e2z,e3z = rot.rot1(masa,xstr,ystr,zstr,vx,vy,vz,r90[-1])
        
        xn=e1x*xgas+e1y*ygas+e1z*zgas
        yn=e2x*xgas+e2y*ygas+e2z*zgas
        zn=e3x*xgas+e3y*ygas+e3z*zgas
##posiciones de partículas que se quiere graficar

        pos=np.ndarray([3,np.size(xn)])
        pos[0,:]=xn
        pos[1,:]=yn
        pos[2,:]=zn

#generador del gráfico
        corte,=np.where((xn<15) & (yn<15) & (zn<15) & (xn>-15) & (yn>-15) & (zn>-15))
        
        particles=sph.Particles(pos[:,corte],masa_gas[corte]*1e10,nb=5)

#escena del gráfico---------
        escena=sph.Scene(particles)

#como queres la escena-----------
        escena.set_autocamera(mode='density')
        rl=15
        
        escena.update_camera(r='infinity',x=0,y=0,z=0,extent=[-rl,rl,-rl,rl], t=90)
#renderizado de la escena (aca hace la grid y cuenta particulas)
        rend=sph.Render(escena)
##extencion del grafico
        extent=escena.get_extent()
#escala logaritmica-----------------
        rend.set_logscale()
#rango que tiene la escala  de colores-----
        vmin=4
        vmax=9
# escala de colores que te guste (http://matplotlib.org/examples/color/colormaps_reference.html)
        cmap='nipy_spectral'
#grafico-----------
        fig=plt.figure(5,figsize=(15,15))
        ax1=fig.add_subplot(221)
        ax1.imshow(rend.get_image(),extent=extent,origin='lower',cmap=cmap)
        plt.ylabel('z[kpc]', fontsize=24)
        ax1.set_title('%s' %name)
        ax1.minorticks_on()
        ax1.tick_params( labelsize=18)
        ax1.tick_params('both', length=3, width=1.2,which='minor')
        ax1.tick_params('both', length=6, width=1.2,which='major')
        
        particles=sph.Particles(pos[:,corte],masa_gas[corte]*1e10,nb=5)
        escena=sph.Scene(particles)
        escena.update_camera(r='infinity',x=0,y=0,z=0,extent=[-rl,rl,-rl,rl], t=90, p=90)
        rend=sph.Render(escena)
        extent=escena.get_extent()
        rend.set_logscale()
        
        ax2=fig.add_subplot(222)
        ax2.imshow(rend.get_image(),extent=extent,origin='lower',cmap=cmap)
        plt.xlabel('y[kpc]', fontsize=24)
        ax2.minorticks_on()
        ax2.tick_params( labelsize=18)
        ax2.tick_params('both', length=3, width=1.2,which='minor')
        ax2.tick_params('both', length=6, width=1.2,which='major')
        
        particles=sph.Particles(pos[:,corte],masa_gas[corte]*1e10,nb=5)
        escena=sph.Scene(particles)
        escena.update_camera(r='infinity',x=0,y=0,z=0,extent=[-rl,rl,-rl,rl])
        rend=sph.Render(escena)
        extent=escena.get_extent()
        rend.set_logscale()

        ax3=fig.add_subplot(223)
        ax3.imshow(rend.get_image(),extent=extent,origin='lower',cmap=cmap)
        plt.xlabel('x[kpc]', fontsize=24)
        plt.ylabel('y[kpc]', fontsize=24)
        ax3.minorticks_on()
        ax3.tick_params( labelsize=18)
        ax3.tick_params('both', length=3, width=1.2,which='minor')
        ax3.tick_params('both', length=6, width=1.2,which='major')
        plt.text(6, 12.5,str('%.3f'%t)+'Gyr', fontsize=18, color='white')

        fig.savefig('/home/ornela/python/gas/'+'%s' %name +'/'+str('%03d'%isnap)+'-'+'%s'%name+'.png', dpi = 300)
        
        plt.clf()
        #plt.show()

30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
29

In [4]:
#Graficamos las gas face-on con el sphviewer
import matplotlib.pyplot as plt
import h5py
import rotation as rot
import sphviewer as sph


h=0.732
a0=1
omega_lambda=0.716
omega_matter=0.1277/(h**2.)

snapshot=range(31,32)
for isnap in snapshot:
    snap = h5py.File('/home/ornela/python/ornella/outputs_1/snap_'+str('%03d'%isnap)+'.h5py', 'r')

    a = snap['subhalo_000/Time'].value
    z = (a0/a)-1
    t = conv(z, h, omega_lambda, omega_matter)
    
    print isnap
    #for name in snap:
    pstr = snap['subhalo_021/Str/Coordinates'].value
    cm   = snap['subhalo_021/Center'].value
    vel  = snap['subhalo_021/Str/Velocities'].value
    masa = snap['subhalo_021/Str/Masses'].value
        
    pgas = snap['subhalo_021/Gas/Coordinates'].value
    masa_gas = snap['subhalo_021/Gas/Masses'].value
#aca paso las coordenadas respecto al centro de la galaxia
    xstr = (pstr[:,0]-cm[0])/h
    ystr = (pstr[:,1]-cm[1])/h
    zstr = (pstr[:,2]-cm[2])/h
    r = np.sqrt(xstr**2+ystr**2+zstr**2)
    masa = masa/h
        
    xgas = (pgas[:,0]-cm[0])/h
    ygas = (pgas[:,1]-cm[1])/h
    zgas = (pgas[:,2]-cm[2])/h
    r_gas = np.sqrt(xstr**2+ystr**2+zstr**2)
    masa_gas = masa_gas/h
        
    limit = np.where(r<20)
    r_sort = np.sort(r[limit])
    r_indice = np.argsort(r[limit])
                
    Mc = np.cumsum((masa[limit])[r_indice])
    Mtot = Mc[-1]
               
    razon = Mc/Mtot
    noventa, = np.where(razon < 0.9)
    cincuenta, = np.where(razon < 0.5)
        
    r90 = r_sort[noventa]
    r50 = r_sort[cincuenta]
        
#esto sirve si quiero hacer una submuestra de mi conjunto de datos
    veloc,=np.where(r<r50[-1])
#escribo las componentes de la velocidad del centro de masa
    vxcm = sum(masa[veloc]*vel[veloc,0])/sum(masa[veloc])
    vycm = sum(masa[veloc]*vel[veloc,1])/sum(masa[veloc])
    vzcm = sum(masa[veloc]*vel[veloc,2])/sum(masa[veloc])
#saco las velocidades de las estrellas respecto del centro de masa de la galaxia
    vx = vel[:,0] - vxcm
    vy = vel[:,1] - vycm
    vz = vel[:,2] - vzcm
    e1x,e2x,e3x,e1y,e2y,e3y,e1z,e2z,e3z = rot.rot1(masa,xstr,ystr,zstr,vx,vy,vz,r90[-1])
        
    xn=e1x*xgas+e1y*ygas+e1z*zgas
    yn=e2x*xgas+e2y*ygas+e2z*zgas
    zn=e3x*xgas+e3y*ygas+e3z*zgas
##posiciones de partículas que se quiere graficar

    pos=np.ndarray([3,np.size(xn)])
    pos[0,:]=xn
    pos[1,:]=yn
    pos[2,:]=zn

#generador del gráfico
    corte,=np.where((xn<15) & (yn<15) & (zn<15) & (xn>-15) & (yn>-15) & (zn>-15))
      
    particles=sph.Particles(pos[:,corte],masa_gas[corte]*1e10,nb=5)

#escena del gráfico---------
    escena=sph.Scene(particles)

#como queres la escena-----------
    escena.set_autocamera(mode='density')
    rl=15
       
    escena.update_camera(r='infinity',x=0,y=0,z=0,extent=[-rl,rl,-rl,rl], t=90)
#renderizado de la escena (aca hace la grid y cuenta particulas)
    rend=sph.Render(escena)
##extencion del grafico
    extent=escena.get_extent()
#escala logaritmica-----------------
    rend.set_logscale()
#rango que tiene la escala  de colores-----
    vmin=4
    vmax=9
# escala de colores que te guste (http://matplotlib.org/examples/color/colormaps_reference.html)
    cmap='nipy_spectral'
#grafico-----------
    fig=plt.figure(5,figsize=(15,15))
    ax1=fig.add_subplot(221)
    ax1.imshow(rend.get_image(),extent=extent,origin='lower',cmap=cmap)
    plt.ylabel('z[kpc]', fontsize=24)
    ax1.set_title('subhalo_021')
    ax1.minorticks_on()
    ax1.tick_params( labelsize=18)
    ax1.tick_params('both', length=3, width=1.2,which='minor')
    ax1.tick_params('both', length=6, width=1.2,which='major')
        
    particles=sph.Particles(pos[:,corte],masa_gas[corte]*1e10,nb=5)
    escena=sph.Scene(particles)
    escena.update_camera(r='infinity',x=0,y=0,z=0,extent=[-rl,rl,-rl,rl], t=90, p=90)
    rend=sph.Render(escena)
    extent=escena.get_extent()
    rend.set_logscale()
        
    ax2=fig.add_subplot(222)
    ax2.imshow(rend.get_image(),extent=extent,origin='lower',cmap=cmap)
    plt.xlabel('y[kpc]', fontsize=24)
    ax2.minorticks_on()
    ax2.tick_params( labelsize=18)
    ax2.tick_params('both', length=3, width=1.2,which='minor')
    ax2.tick_params('both', length=6, width=1.2,which='major')
        
    particles=sph.Particles(pos[:,corte],masa_gas[corte]*1e10,nb=5)
    escena=sph.Scene(particles)
    escena.update_camera(r='infinity',x=0,y=0,z=0,extent=[-rl,rl,-rl,rl])
    rend=sph.Render(escena)
    extent=escena.get_extent()
    rend.set_logscale()

    ax3=fig.add_subplot(223)
    ax3.imshow(rend.get_image(),extent=extent,origin='lower',cmap=cmap)
    plt.xlabel('x[kpc]', fontsize=24)
    plt.ylabel('y[kpc]', fontsize=24)
    ax3.minorticks_on()
    ax3.tick_params( labelsize=18)
    ax3.tick_params('both', length=3, width=1.2,which='minor')
    ax3.tick_params('both', length=6, width=1.2,which='major')
    plt.text(6, 12.5,str('%.3f'%t)+'Gyr', fontsize=18, color='white')

    fig.savefig('/home/ornela/python/gas/subhalo_021' +'/'+str('%03d'%isnap)+'-'+'subhalo_021'+'.png', dpi = 300)
        
    plt.clf()
        #plt.show()

31
